In [31]:
import pandas as pd

In [1]:
from Bio.Blast import NCBIWWW

In [2]:
fasta_string = 'MPEVTGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAADGSPFQLV'
result_handle = NCBIWWW.qblast("blastp", "nr", fasta_string)

In [8]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

In [9]:
E_VALUE_THRESH = 0.04
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')

****Alignment****
('sequence:', u'gi|491667572|ref|WP_005524287.1| anaerobic ribonucleoside-triphosphate reductase activating protein [Corynebacterium matruchotii] >gi|305660510|gb|EFM50007.1| anaerobic ribonucleoside-triphosphate reductase activating protein [Corynebacterium matruchotii ATCC 14266]')
('length:', 225)
('e value:', 3.20819e-45)
MPEVTGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAADGSPFQLV...
MPEVTGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAADGSPFQLV...
MPEVTGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAADGSPFQLV...
****Alignment****
('sequence:', u'gi|491662834|ref|WP_005519550.1| ribonucleoside-triphosphate reductase activating protein [Corynebacterium matruchotii] >gi|224946721|gb|EEG27930.1| anaerobic ribonucleoside-triphosphate reductase activating protein [Corynebacterium matruchotii ATCC 33806]')
('length:', 225)
('e value:', 4.62848e-44)
MPEVTGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAADGSPFQLV...
MP

In [4]:
## now you should run the smith-waterman alignment on the top scoring blast hits (to locally align your protein of interest with the blastp hits)

In [10]:
import swalign

In [12]:
match = 2 
mismatch = -1 
scoring = swalign.NucleotideScoringMatrix(match, mismatch)
sw = swalign.LocalAlignment(scoring)

In [22]:
alignment = sw.align(fasta_string,hsp.match)
alignment.dump()

Query:  5 TGCA   +   WD L LL DAGVD Q+RTT+W  S +++ +  L   V+  G +LV+QQAR  D 68
          ||||.......||.|.||.|||||.|.|||.|..|........|...|...|..||.||||..|
Ref  :  5 TGCAATVSGRVWDSLHLLVDAGVDLQLRTTLWRDSVISQHLPELQHLVSEQGFDLVIQQARAAD 68

Score: 26
Matches: 30 (46.9%)
Mismatches: 34
CIGAR: 64M


In [42]:
read_to_struct = []

E_VALUE_THRESH = 0.04
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            sw_tmp = sw.align(fasta_string,hsp.match)
            score = sw_tmp.score
            desc = alignment.title.split('|')[4]
            ref = alignment.title.split('|')[3]
            id_type = alignment.title.split('|')[2]
            org = alignment.title.split('[')[1].split(']')[0]
            matches = sw_tmp.matches
            read_to_struct.append({'desc':desc, 'id_type':id_type ,'id':ref, 'organism':org, 'sw-score':score,'matches':matches, 'e_val':hsp.expect, 'alignment_length':alignment.length})
DF_alignment = pd.DataFrame(read_to_struct)
DF_alignment.head(2)

,alignment_length,desc,e_val,id,id_type,matches,organism,sw-score
0,225,anaerobic ribonucleoside-triphosphate reducta...,3.208190e-45,WP_005524287.1,ref,75,Corynebacterium matruchotii,150
1,225,ribonucleoside-triphosphate reductase activat...,4.628480e-44,WP_005519550.1,ref,74,Corynebacterium matruchotii,147


In [40]:
# you might need to do a sensitivity check here- to see if the top scoring (sw-score) also has the lowest e val. 

In [41]:
#Choose the top score as the gene id to go on to the next one...